Stage 1: Authenticate GovWin Connection and obtain a access token.

In [1]:
import requests

client_id = 'C9NNU5QHTAFGT3I2V8L51COQ23E71IA9KMNJR3SS05AGI' #"H859MH8H3DN5A03S7N6MU702E3SMCFF6F30E2L40QG58G"
client_secret = '1QS5CR2AB75Q1IFT8LKU8VKLRM8C7UB5NM27AQ821ELK6' #"9NH77FAHV5LKP2MBADA6LL8GLJEPUPTNTVVRLTO74GMQ7"

url = f"https://services.govwin.com/neo-ws/oauth/token?client_id={client_id}&client_secret={client_secret}&grant_type=password&username=webservices4@bah.com&password=Testing05302023&scope=read"

data = {"key1": "value1", "key2": "value2"}

response = requests.post(url, data=data)

# Check the response status code
if response.status_code == 200:
    print("POST request was successful, connection established.")

    access_token = response.json().get("access_token")
    print("Bearer token:", access_token)
    
else:
    print("POST request failed. Status code:", response.status_code)

# Print the response content
# print(response.text)


POST request was successful, connection established.
Bearer token: 131f6e44-1d9d-42f6-875e-7ea39a62782a


Stage 2: Access saved search metadata.

In [2]:
# Set the bearer token
bearer_token = access_token

endpoint  = "https://services.govwin.com/neo-ws/opportunities?savedSearchId=1236146"

# Make the API request
headers = {"Authorization": f"Bearer {bearer_token}"}

# Get first 100 opportunities
response = requests.get(endpoint, headers=headers, params={'oppSelectionDateFrom': '01/01/1900', 'max':100, 'offset':0})

# Check the response status code
if response.status_code == 200:
    print("GET request was successful, endpoint collection saved to artifacts folder.")
else:
    print("GET request failed. Status code:", response.status_code)


#save response.text to a text file
with open(f'artifacts/govwin_extract.json', 'w') as file:
    file.write(response.text)


#save response.text to a text file
with open(f'artifacts/govwin_extract.txt', 'w') as file:
    file.write(response.text)


GET request was successful, endpoint collection saved to artifacts folder.


In [3]:
# Get remaining 29 opportunities
# Get first 100 opportunities
response = requests.get(endpoint, headers=headers, params={'oppSelectionDateFrom': '01/01/1900', 'max':100, 'offset':100})

# Check the response status code
if response.status_code == 200:
    print("GET request was successful, endpoint collection saved to artifacts folder.")
else:
    print("GET request failed. Status code:", response.status_code)


#save response.text to a text file
with open(f'artifacts/govwin_extract.json', 'a') as file:
    file.write(response.text)


#save response.text to a text file
with open(f'artifacts/govwin_extract.txt', 'a') as file:
    file.write(response.text)


GET request was successful, endpoint collection saved to artifacts folder.


In [4]:
import sys


with open('artifacts/govwin_extract.txt') as file:
    raw_text = file.read().split()
    opp_id_list = []
    for line in raw_text:

        if "iqOppId" in line:
            line_arr = line.split(',')

            for i in range(len(line_arr)):

                if "iqOppId" in line_arr[i]:
                     
                     opp_id = line_arr[i].split(':')[1]
                     opp_id_list.append(opp_id)

    # write to text file
    with open('artifacts/govwin_oppids.txt', 'w') as fp:
        fp.write("%s" % opp_id_list)

    print('Done')


Done


In [5]:
endpoint  = "https://services.govwin.com/neo-ws/opportunities/OPP178753"
response = requests.get(endpoint, headers=headers)

# Check the response status code
if response.status_code == 200:
    print("GET request was successful, endpoint collection saved to artifacts folder.")
else:
    print("GET request failed. Status code:", response.status_code)

# save response.text to a text file
with open(f'artifacts/govwin_OPP178753.txt', 'a') as file:
    file.write(response.text)

GET request was successful, endpoint collection saved to artifacts folder.


Stage 3: JSON search and feature extraction.

In [6]:
import json

with open('search_items.txt', 'r') as file:
    search_items = [line.strip() for line in file.readlines()]

# load json file
with open('artifacts/govwin_extract.json', 'r') as file:
    data = json.load(file)

# scrape desired information based on search items txt file
scraped_data = {}
for item in search_items:
    if item in data:
        scraped_data[item] = data['opportunities'][0][item]

#print scraped data
for item, value in scraped_data.items():
    print(f"{item}: {value}")

#save scraped data to a json file
with open(f'artifacts/govwin_extract_scraped.txt', 'w') as file:
    json.dump(scraped_data, file, indent=4)

JSONDecodeError: Extra data: line 1 column 878193 (char 878192)

In [7]:
import json

# Read the search items from the text file
with open('search_items.txt', 'r') as file:
    search_items = [line.strip() for line in file.readlines()]

# Load the JSON data
with open('artifacts\govwin_extract.json', 'r') as file:
    data = json.load(file)

# Scrape the desired information based on the search items
scraped_data = {}
opportunity = data['opportunities'][0]

for item in search_items:
    if item in opportunity:
        scraped_data[item] = opportunity[item]
    elif item in opportunity['govEntity']:
        scraped_data[item] = opportunity['govEntity'][item]
    elif item in opportunity['primaryNAICS']:
        scraped_data[item] = opportunity['primaryNAICS'][item]

# Print the scraped data
for item, value in scraped_data.items():
    print(f"{item}: {value}")



JSONDecodeError: Extra data: line 1 column 878193 (char 878192)